# Bibliotecas

In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Leitura dos Dados

In [2]:
endereco = '../BaseDadosSelecionados/'
# endereco2 = 'Agrupado_Ocorrencia'
nomeArq = 'ES_datatran2019_Apriori'

In [3]:
data = pd.read_csv(endereco + nomeArq + '.csv', encoding='ISO-8859-1', sep=';')

In [4]:
data.shape

(2841, 21)

In [5]:
data.head()

,dia_semana,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,...,tracado_via,uso_solo,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos
0,dia_semana=terça-feira,br=262.0,km=95,municipio=DOMINGOS MARTINS,causa_acidente=Não guardar distância de segurança,tipo_acidente=Engavetamento,classificacao_acidente=Sem Vítimas,fase_dia=Pleno dia,sentido_via=Decrescente,condicao_metereologica=Céu Claro,...,tracado_via=Reta,uso_solo=1,pessoas=4,mortos=0,feridos_leves=0,feridos_graves=0,ilesos=4,ignorados=0,feridos=0,veiculos=4
1,dia_semana=terça-feira,br=101.0,km=77.1,municipio=SAO MATEUS,causa_acidente=Falta de Atenção à Condução,tipo_acidente=Colisão traseira,classificacao_acidente=Com Vítimas Feridas,fase_dia=Pleno dia,sentido_via=Decrescente,condicao_metereologica=Céu Claro,...,tracado_via=Desvio Temporário,uso_solo=0,pessoas=3,mortos=0,feridos_leves=1,feridos_graves=1,ilesos=1,ignorados=0,feridos=2,veiculos=2
2,dia_semana=terça-feira,br=101.0,km=299.2,municipio=VIANA,causa_acidente=Desobediência às normas de trân...,tipo_acidente=Colisão transversal,classificacao_acidente=Sem Vítimas,fase_dia=Anoitecer,sentido_via=Decrescente,condicao_metereologica=Céu Claro,...,tracado_via=Não Informado,uso_solo=1,pessoas=3,mortos=0,feridos_leves=0,feridos_graves=0,ilesos=3,ignorados=0,feridos=0,veiculos=2
3,dia_semana=terça-feira,br=262.0,km=168,municipio=IBATIBA,causa_acidente=Falta de Atenção à Condução,tipo_acidente=Queda de ocupante de veículo,classificacao_acidente=Com Vítimas Feridas,fase_dia=Anoitecer,sentido_via=Crescente,condicao_metereologica=Céu Claro,...,tracado_via=Reta,uso_solo=0,pessoas=1,mortos=0,feridos_leves=0,feridos_graves=1,ilesos=0,ignorados=0,feridos=1,veiculos=1
4,dia_semana=terça-feira,br=101.0,km=329,municipio=GUARAPARI,causa_acidente=Ingestão de Álcool,tipo_acidente=Colisão lateral,classificacao_acidente=Com Vítimas Feridas,fase_dia=Plena Noite,sentido_via=Crescente,condicao_metereologica=Céu Claro,...,tracado_via=Reta,uso_solo=0,pessoas=4,mortos=0,feridos_leves=3,feridos_graves=0,ilesos=0,ignorados=1,feridos=3,veiculos=2


# Carregamento dos dados em uma matriz

In [6]:
# data.shape[0] LINHAS e data.shape[1] COLUNAS
dataset = []
for i in range(0, data.shape[0]):
    dataset.append([str(data.values[i,j]) for j in range(0, data.shape[1])])

## Organização dos dados

In [7]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
# df

In [8]:
oht_ary = te.fit(dataset).transform(dataset, sparse=True)
sparse_df = pd.DataFrame.sparse.from_spmatrix(oht_ary, columns=te.columns_)
# sparse_df

# Execução do algoritmo Apriori

In [9]:
numMinSupport = 0.2

In [10]:
%%time
apriori(sparse_df, min_support=numMinSupport, use_colnames=True, verbose=1)

Processing 184 combinations | Sampling itemset size 875
Wall time: 3.22 s


,support,itemsets
0,0.714537,(br=101.0)
1,0.200986,(br=262.0)
2,0.404083,(causa_acidente=Falta de Atenção à Condução)
3,0.829285,(classificacao_acidente=Com Vítimas Feridas)
4,0.588525,(condicao_metereologica=Céu Claro)
...,...,...
1713,0.215065,"(feridos_leves=1, fase_dia=Pleno dia, feridos_..."
1714,0.220697,"(sentido_via=Crescente, feridos_leves=1, ferid..."
1715,0.233017,"(feridos_leves=1, tracado_via=Reta, feridos_gr..."
1716,0.210841,"(feridos_leves=1, feridos_graves=0, feridos=1,..."


In [11]:
%%time
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df, min_support=numMinSupport, use_colnames=True)

# frequent_itemsets

Wall time: 2.03 s


# Salvando os resultados em um .csv

In [12]:
association_rules_saida = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

In [13]:
saidaRecords = pd.DataFrame(association_rules_saida)

In [14]:
saidaRecords.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(causa_acidente=Falta de Atenção à Condução),(br=101.0),0.404083,0.714537,0.289335,0.716028,1.002086,0.000602,1.005250
1,(br=101.0),(classificacao_acidente=Com Vítimas Feridas),0.714537,0.829285,0.598381,0.837438,1.009831,0.005826,1.050153
2,(classificacao_acidente=Com Vítimas Feridas),(br=101.0),0.829285,0.714537,0.598381,0.721562,1.009831,0.005826,1.025229
3,(condicao_metereologica=Céu Claro),(br=101.0),0.588525,0.714537,0.422738,0.718301,1.005268,0.002215,1.013363
4,(fase_dia=Plena Noite),(br=101.0),0.355861,0.714537,0.255192,0.717112,1.003603,0.000916,1.009101
5,(fase_dia=Pleno dia),(br=101.0),0.532911,0.714537,0.378740,0.710700,0.994630,-0.002045,0.986737
6,(feridos=1),(br=101.0),0.606477,0.714537,0.442802,0.730122,1.021811,0.009452,1.057747
7,(br=101.0),(feridos_graves=0),0.714537,0.690954,0.506864,0.709360,1.026638,0.013152,1.063328
8,(feridos_graves=0),(br=101.0),0.690954,0.714537,0.506864,0.733571,1.026638,0.013152,1.071441
9,(feridos_leves=1),(br=101.0),0.460049,0.714537,0.339669,0.738332,1.033301,0.010947,1.090936


In [15]:
association_rules_saida.to_csv('AprioriMlxtend_'+ nomeArq + '.csv', encoding= 'ISO-8859-1', sep= ';', index=False)

In [16]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(br=101.0),(municipio=SERRA),0.714537,0.222809,0.222809,0.311823,1.399507,0.063604,1.129347
1,(municipio=SERRA),(br=101.0),0.222809,0.714537,0.222809,1.000000,1.399507,0.063604,inf
2,(feridos_leves=1),(feridos=1),0.460049,0.606477,0.411123,0.893650,1.473510,0.132114,3.700252
3,(feridos=1),(feridos_leves=1),0.606477,0.460049,0.411123,0.677887,1.473510,0.132114,1.676279
4,(feridos_leves=1),(feridos_graves=0),0.460049,0.690954,0.411123,0.893650,1.293356,0.093250,2.905923
...,...,...,...,...,...,...,...,...,...
8677,(veiculos=2),"(pessoas=2, feridos=1, classificacao_acidente=...",0.529039,0.266807,0.227385,0.429807,1.610926,0.086233,1.285867
8678,(pessoas=2),"(veiculos=2, feridos=1, classificacao_acidente...",0.462513,0.227385,0.227385,0.491629,2.162100,0.122216,1.519785
8679,(feridos=1),"(veiculos=2, pessoas=2, classificacao_acidente...",0.606477,0.227385,0.227385,0.374927,1.648868,0.089481,1.236041
8680,(classificacao_acidente=Com Vítimas Feridas),"(veiculos=2, pessoas=2, feridos=1, ilesos=1, m...",0.829285,0.227385,0.227385,0.274194,1.205857,0.038818,1.064492


## Método para contabilizaro número de consequentes e antecedentes em cada regra

In [17]:
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(br=101.0),(municipio=SERRA),0.714537,0.222809,0.222809,0.311823,1.399507,0.063604,1.129347,1
1,(municipio=SERRA),(br=101.0),0.222809,0.714537,0.222809,1.000000,1.399507,0.063604,inf,1
2,(feridos_leves=1),(feridos=1),0.460049,0.606477,0.411123,0.893650,1.473510,0.132114,3.700252,1
3,(feridos=1),(feridos_leves=1),0.606477,0.460049,0.411123,0.677887,1.473510,0.132114,1.676279,1
4,(feridos_leves=1),(feridos_graves=0),0.460049,0.690954,0.411123,0.893650,1.293356,0.093250,2.905923,1
...,...,...,...,...,...,...,...,...,...,...
8677,(veiculos=2),"(pessoas=2, feridos=1, classificacao_acidente=...",0.529039,0.266807,0.227385,0.429807,1.610926,0.086233,1.285867,1
8678,(pessoas=2),"(veiculos=2, feridos=1, classificacao_acidente...",0.462513,0.227385,0.227385,0.491629,2.162100,0.122216,1.519785,1
8679,(feridos=1),"(veiculos=2, pessoas=2, classificacao_acidente...",0.606477,0.227385,0.227385,0.374927,1.648868,0.089481,1.236041,1
8680,(classificacao_acidente=Com Vítimas Feridas),"(veiculos=2, pessoas=2, feridos=1, ilesos=1, m...",0.829285,0.227385,0.227385,0.274194,1.205857,0.038818,1.064492,1


In [18]:
rules["consequents_len"] = rules["consequents"].apply(lambda x: len(x))
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequents_len
0,(br=101.0),(municipio=SERRA),0.714537,0.222809,0.222809,0.311823,1.399507,0.063604,1.129347,1,1
1,(municipio=SERRA),(br=101.0),0.222809,0.714537,0.222809,1.000000,1.399507,0.063604,inf,1,1
2,(feridos_leves=1),(feridos=1),0.460049,0.606477,0.411123,0.893650,1.473510,0.132114,3.700252,1,1
3,(feridos=1),(feridos_leves=1),0.606477,0.460049,0.411123,0.677887,1.473510,0.132114,1.676279,1,1
4,(feridos_leves=1),(feridos_graves=0),0.460049,0.690954,0.411123,0.893650,1.293356,0.093250,2.905923,1,1
...,...,...,...,...,...,...,...,...,...,...,...
8677,(veiculos=2),"(pessoas=2, feridos=1, classificacao_acidente=...",0.529039,0.266807,0.227385,0.429807,1.610926,0.086233,1.285867,1,6
8678,(pessoas=2),"(veiculos=2, feridos=1, classificacao_acidente...",0.462513,0.227385,0.227385,0.491629,2.162100,0.122216,1.519785,1,6
8679,(feridos=1),"(veiculos=2, pessoas=2, classificacao_acidente...",0.606477,0.227385,0.227385,0.374927,1.648868,0.089481,1.236041,1,6
8680,(classificacao_acidente=Com Vítimas Feridas),"(veiculos=2, pessoas=2, feridos=1, ilesos=1, m...",0.829285,0.227385,0.227385,0.274194,1.205857,0.038818,1.064492,1,6


### Filtro para facilitar a visualização dos resultados

In [19]:
rules[ (rules['antecedent_len'] >= 2) &
       (rules['confidence'] > 0.75) &
       (rules['lift'] > 1.2) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequents_len
26,"(feridos_leves=1, br=101.0)",(feridos=1),0.339669,0.606477,0.308342,0.907772,1.496796,0.102341,4.266855,2,1
30,"(feridos_leves=1, br=101.0)",(feridos_graves=0),0.339669,0.690954,0.308342,0.907772,1.313795,0.073646,3.350893,2,1
38,"(br=101.0, ilesos=1)",(veiculos=2),0.309046,0.529039,0.234424,0.758542,1.433811,0.070927,1.950489,2,1
43,"(municipio=SERRA, mortos=0)",(br=101.0),0.215417,0.714537,0.215417,1.000000,1.399507,0.061494,inf,2,1
51,"(br=101.0, uso_solo=0)",(tipo_pista=Simples),0.297430,0.525167,0.226329,0.760947,1.448961,0.070128,1.986306,2,1
...,...,...,...,...,...,...,...,...,...,...,...
8646,"(ilesos=1, feridos=1, classificacao_acidente=C...","(pessoas=2, veiculos=2, ignorados=0, mortos=0)",0.290743,0.296023,0.227385,0.782082,2.641969,0.141318,3.230474,3,4
8649,"(ilesos=1, feridos=1, mortos=0)","(pessoas=2, ignorados=0, veiculos=2, classific...",0.290743,0.252728,0.227385,0.782082,3.094563,0.153906,3.429149,3,4
8650,"(ilesos=1, feridos=1, ignorados=0)","(pessoas=2, veiculos=2, classificacao_acidente...",0.267863,0.281943,0.227385,0.848883,3.010832,0.151863,4.751664,3,4
8664,"(pessoas=2, ilesos=1)","(veiculos=2, feridos=1, classificacao_acidente...",0.286167,0.265751,0.227385,0.794588,2.989966,0.151335,3.574515,2,5


In [20]:
# rules[rules['antecedents'] == {'Eggs', 'Kidney Beans'}]

# Nova execução do algoritmo apriori - Apresentação diferente

In [21]:
apriori(df, min_support=numMinSupport)

,support,itemsets
0,0.714537,(0)
1,0.200986,(2)
2,0.404083,(19)
3,0.829285,(33)
4,0.588525,(36)
...,...,...
1713,0.215065,"(1440, 33, 66, 74, 84, 53, 55)"
1714,0.220697,"(1440, 33, 66, 1504, 74, 84, 55)"
1715,0.233017,"(1440, 33, 66, 74, 84, 55, 1530)"
1716,0.210841,"(1440, 33, 66, 74, 84, 55, 1535)"


In [22]:
%%time
apriori(df, min_support=numMinSupport, use_colnames=True)

Wall time: 2.19 s


,support,itemsets
0,0.714537,(br=101.0)
1,0.200986,(br=262.0)
2,0.404083,(causa_acidente=Falta de Atenção à Condução)
3,0.829285,(classificacao_acidente=Com Vítimas Feridas)
4,0.588525,(condicao_metereologica=Céu Claro)
...,...,...
1713,0.215065,"(feridos_leves=1, fase_dia=Pleno dia, feridos_..."
1714,0.220697,"(sentido_via=Crescente, feridos_leves=1, ferid..."
1715,0.233017,"(feridos_leves=1, tracado_via=Reta, feridos_gr..."
1716,0.210841,"(feridos_leves=1, feridos_graves=0, feridos=1,..."


In [23]:
frequent_itemsets = apriori(df, min_support=numMinSupport, use_colnames=True, verbose=1)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

Processing 184 combinations | Sampling itemset size 875


,support,itemsets,length
0,0.714537,(br=101.0),1
1,0.200986,(br=262.0),1
2,0.404083,(causa_acidente=Falta de Atenção à Condução),1
3,0.829285,(classificacao_acidente=Com Vítimas Feridas),1
4,0.588525,(condicao_metereologica=Céu Claro),1
...,...,...,...
1713,0.215065,"(feridos_leves=1, fase_dia=Pleno dia, feridos_...",7
1714,0.220697,"(sentido_via=Crescente, feridos_leves=1, ferid...",7
1715,0.233017,"(feridos_leves=1, tracado_via=Reta, feridos_gr...",7
1716,0.210841,"(feridos_leves=1, feridos_graves=0, feridos=1,...",7


## Filtro referente ao último método apresentado

In [24]:
frequent_itemsets[ (frequent_itemsets['length'] == 2) &
                   (frequent_itemsets['support'] >= 0.8) ]

,support,itemsets,length
76,0.829285,"(classificacao_acidente=Com Vítimas Feridas, m...",2
176,0.832101,"(ignorados=0, mortos=0)",2


In [25]:
# frequent_itemsets[ frequent_itemsets['itemsets'] == {'Onion', 'Eggs'} ]